---

This notebook contains the exploratory data analysis (EDA) for the first module of the **"Intelligent System for Supply Chain Management"** project. 

The main objective is to optimize inventory and purchasing management, with a target of **reducing overstocking by 20%** within 6 months.

- Target Variable for Inventory Optimization: **Stock_Quantity**
- Target Variable for Demand Forecasting: **Sales_Volume**

---

Import Libraries

In [7]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests, acf, pacf

# Configure display and graphs
pd.set_option('display.max_columns', None)
pio.templates.default = "plotly_white"

import warnings
warnings.filterwarnings('ignore')

Load Data

In [8]:
# Define data paths
data_path = os.path.join('../data', 'processed')

# Load Pickle file
read_data = pd.read_pickle(data_path + '/grocery.pkl')

In [9]:
# Sort DataFrame by Date_Received in ascending order
read_data = read_data.sort_values(by='Date_Received').reset_index(drop=True)


# **Target Variables Analysis**
## **1. Stock Quantity**
- **Definition:** Total inventory units held per day across all products
- **Pattern:** Expected to show regular replenishment cycles and consumption trends
- **Key Insight:** Critical for inventory optimization and supply chain planning

## **2. Sales Volume**  
- **Definition:** Total daily units sold across all products
- **Pattern:** Likely exhibits weekly seasonality and potential promotional spikes
- **Key Insight:** Primary driver for demand forecasting and business performance analysis

In [10]:
# These are the key metrics the model aims to predict.
target_columns = ['Stock_Quantity', 'Sales_Volume']

# These are predictors that are fixed or change very infrequently for each product.
stat_predict = ['Product_Name','Category', 'Supplier_Name', 'Reorder_Level', 'Reorder_Quantity']

# These are predictors that can change over time for each product.
dyn_predict = ['Unit_Price', 'Date_Received', 'Inventory_Turnover_Rate', 'Status', 'Days_For_Expiration', 'Expiration_Status', 'DOI_Inv_Turnover','Delivery_Lag']

# The complete set of all columns used in the analysis.
all_columns = stat_predict + dyn_predict + target_columns

In [11]:
# Create a copy of the selected columns from the original DataFrame for time series analysis
df_ts = read_data[all_columns].copy()

In [12]:
# data Information
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Product_Name             990 non-null    object        
 1   Category                 990 non-null    category      
 2   Supplier_Name            990 non-null    object        
 3   Reorder_Level            990 non-null    int64         
 4   Reorder_Quantity         990 non-null    int64         
 5   Unit_Price               990 non-null    float64       
 6   Date_Received            990 non-null    datetime64[ns]
 7   Inventory_Turnover_Rate  990 non-null    int64         
 8   Status                   990 non-null    category      
 9   Days_For_Expiration      990 non-null    Int64         
 10  Expiration_Status        990 non-null    category      
 11  DOI_Inv_Turnover         990 non-null    int64         
 12  Delivery_Lag             990 non-nul

In [13]:
# Aggregate target columns by date, summing values for each day
data = read_data.groupby('Date_Received')[target_columns].sum().reset_index()

# Set the date column as the DataFrame index for time series operations
df_target = data.set_index('Date_Received')

# Resample to daily frequency, ensuring all dates are represented and filling missing days with 0
df_target = df_target.resample('D').sum().fillna(0)

### Series Visualization

In [14]:
# Create a blank subplot figure with 2 vertical panels and titles
fig = make_subplots(rows=2, cols=1, subplot_titles=['Sales Volume', 'Stock Quantity'])

# Create individual line plots for each target variable
fig1 = px.line(df_target, y=['Sales_Volume'], color_discrete_sequence=['blue'])
fig2 = px.line(df_target, y=['Stock_Quantity'], color_discrete_sequence=['green'])

# Add the first plot (Sales Volume) to the top panel
fig.add_trace(fig1.data[0], row=1, col=1)
# Add the second plot (Stock Quantity) to the bottom panel
fig.add_trace(fig2.data[0], row=2, col=1)

# Adjust the overall figure layout (hide legend, set height)
fig.update_layout(showlegend=False, height=600)
# Display the interactive plot
fig.show()

#### **Summary:**

A time series analysis of Sales Volume and Stock Quantity revealed the following key points:
*   **Stationarity:** Both series appear to be mean-stationary, showing no consistent long-term upward or downward trend.
*   **Seasonality:** A **weekly seasonal pattern** was identified, especially in sales, with peaks and valleys likely corresponding to days of the week (e.g., higher weekend sales).
*   **Autocorrelation:** The weekly seasonality indicates strong autocorrelation, meaning a day's value is correlated with values from the same day in previous weeks.
*   **Volatility & Outliers:** Both series are highly volatile. A significant outlier was noted in **November 2024**, likely caused by a one-time event like a seasonal promotion.

---

# **Data Structure Prepared**
- **Frequency:** Daily time series (resampled to 'D')
- **Index:** DateTime index (`Date_Received`)
- **Missing Values:** Zero-filled for days without recorded activity
- **Decomposition:** Additive model with 7-day period for weekly seasonality analysis

### Decomposition for Sales Volume and Stock Quantity

In [15]:
# Use an appropriate period, such as 7 for daily data with weekly seasonality
result_sales = seasonal_decompose(df_target['Sales_Volume'], model='additive', period=7)
result_stock = seasonal_decompose(df_target['Stock_Quantity'], model='additive', period=7)

# Create a new DataFrame with the decomposed components
decomposed_df = pd.DataFrame({
    'Trend_Sales': result_sales.trend,
    'Seasonality_Sales': result_sales.seasonal,
    'Residuals_Sales': result_sales.resid,
    'Trend_Stock': result_stock.trend,
    'Seasonality_Stock': result_stock.seasonal,
    'Residuals_Stock': result_stock.resid,
}, index=df_target.index)

# Plot each component individually with Plotly Express
fig_decomposed = make_subplots(rows=2, cols=1, 
                              subplot_titles=['Seasonal Decompose Sales Volume', 
                                            'Seasonal Decompose Stock Quantity'])

# Create individual plots for sales components
fig_decomposed1 = px.line(decomposed_df[['Trend_Sales', 'Seasonality_Sales', 'Residuals_Sales']])
# Create individual plots for stock components with custom colors
fig_decomposed2 = px.line(decomposed_df[['Trend_Stock', 'Seasonality_Stock', 'Residuals_Stock']], 
                         color_discrete_sequence=["#ffaf3c", "#3c5587", "#adb7c4"])

# Add sales components to top subplot
for trace in fig_decomposed1.data[:3]:
    fig_decomposed.add_trace(trace, row=1, col=1)
    
# Add stock components to bottom subplot    
for trace2 in fig_decomposed2.data[:3]:
    fig_decomposed.add_trace(trace2, row=2, col=1)

# Update layout settings
fig_decomposed.update_layout(showlegend=True, height=800)

# Display the interactive decomposition plot
fig_decomposed.show()

#### **Summary**

This analysis compares the decomposition of daily sales volume and stock quantity over a period from March 2024 to January 2025.

**Sales Volume:**
*   **Trend (Blue Line):** No clear long-term growth or decline. The average sales volume remained relatively stable.
*   **Seasonality (Red Line):** Almost flat, indicating **no strong seasonal pattern**. Daily sales fluctuations are random, not tied to specific days or seasons.
*   **Residuals (Green Line):** Shows high fluctuation, closely resembling raw data. This means most sales variability is caused by **random, unpredictable factors**.

**Stock Quantity:**
*   **Trend (Orange Line):** Similar to sales, it oscillates around a central level, showing overall stable average stock.
*   **Seasonality (Black Line):** Shows **clear, repeating seasonal patterns** with significant peaks and valleys (e.g., stocking up before high-sales periods).
*   **Residuals (Gray Line):** Has fluctuations but they are less extreme than for sales, indicating stock is **more predictable** and less influenced by randomness.

**General Conclusion:**
The key difference is that **stock quantity has clear seasonality**, while **sales volume is dominated by random factors**. This makes daily sales unpredictable and a challenge for supply chain planning. In contrast, the predictable pattern in stock is a positive sign, indicating that inventory management can (or already does) account for these recurring patterns, facilitating easier planning and optimization.

---

### Autocorrelation Analysis

In [16]:
# Function for interactive ACF/PACF charts
def plot_acf_pacf(series, title, max_lag=40):
    acf_values = acf(series.dropna(), nlags=max_lag)
    pacf_values = pacf(series.dropna(), nlags=max_lag)
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f'ACF - {title}', f'PACF - {title}'])
    
    # ACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=acf_values, name='ACF'), row=1, col=1)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    
    # PACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=pacf_values, name='PACF'), row=1, col=2)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    
    fig.update_layout(height=400, showlegend=False, title_text=f"Autocorrelation Analysis - {title}")
    fig.show()

# Plot for Sales Volume
plot_acf_pacf(df_target['Sales_Volume'], 'Sales Volume')

# Plot for Stock Quantity
plot_acf_pacf(df_target['Stock_Quantity'], 'Stock Quantity')

#### **Summary:**

A comparative analysis of the **Sales Volume** and **Stock Quantity** time series shows very similar statistical properties:
*   **Autocorrelation:** Both series exhibit **strong first-order autocorrelation** (a significant spike at Lag 1 in the PACF plots), meaning today's value is directly influenced by yesterday's value.
*   **Seasonality:** The ACF and PACF tests did **not** reveal significant seasonal patterns. Any weekly pattern observed visually is too weak to be statistically impactful.
*   **Modeling Implication:** An **AR(1) model** is a suitable starting point for forecasting both series, as it is designed to capture this short-term, one-lag dependency.
*   **Challenge:** Most of the variability in both series is random noise, which will make accurate forecasting difficult.

---

In [17]:
# Create new columns with previous day's values (lag of 1)
df_target['Sales_Volume_Lag1'] = df_target['Sales_Volume'].shift(1)
df_target['Stock_Quantity_Lag1'] = df_target['Stock_Quantity'].shift(1)

# Create subplot figure with 2 horizontal panels
fig_lag = make_subplots(rows=1, cols=2, 
                       subplot_titles=['Autocorrelation Sales Volume', 
                                     'Autocorrelation Stock Quantity'])

# Create scatter plots to visualize autocorrelation
fig_lag1 = px.scatter(df_target, x='Sales_Volume', y='Sales_Volume_Lag1',
                     title='Autocorrelation (Sales_Volume vs. Previous Day Sales)')
fig_lag2 = px.scatter(df_target, x='Stock_Quantity', y='Stock_Quantity_Lag1',
                     title='Autocorrelation (Stock_Quantity vs. Previous Day Stock)', 
                     color_discrete_sequence=['green'])

# Add plots to subfigure
fig_lag.add_trace(fig_lag1.data[0], row=1, col=1)
fig_lag.add_trace(fig_lag2.data[0], row=1, col=2)

# Adjust layout and display
fig_lag.update_layout(showlegend=False, height=600)
fig_lag.show()

#### **Summary:**

An analysis of autocorrelation scatter plots reveals two distinct patterns:
1.  **Sales Volume:** The points are randomly scattered with **no clear pattern**, indicating **no autocorrelation**. Today's sales volume is not a good predictor of tomorrow's, suggesting the data is random or influenced more by other factors like seasonality or promotions.
2.  **Stock Quantity:** The points show a **slight negative trend**. As stock on a given day increases, stock on the next day tends to decrease. This expected supply chain behavior reflects inventory being consumed over time. However, the considerable dispersion indicates this relationship is not perfect and is also influenced by other factors like stock replenishment.

**In short:** Sales are unpredictable day-to-day, while stock follows a more predictable pattern of consumption.

---


### Outliers

In [18]:
# Create a box plot for Sales_Volume and Stock_Quantity
fig_box = px.box(df_target, 
                 y=['Sales_Volume', 'Stock_Quantity'], 
                 title='Distribution of Sales Volume and Stock Quantity with Outlier Detection')
fig_box.update_layout(height=600)
fig_box.show()

#### **Summary:**

Boxplot analysis of **Sales Volume** and **Stock Quantity** confirms:
*   **Similar Distribution:** Both series have a very similar spread and interquartile range (IQR). Most data is concentrated in a central range (Sales: ~80-220 units, Stock: ~75-200 units).
*   **Presence of Outliers:** Both boxplots clearly show multiple outliers, with the most prominent one near **600 units** in each series. This confirms the extreme peak previously identified in the time series plots.

**Conclusion:** The data contains significant outliers that require special attention, as they deviate substantially from the normal data range.

---

### Casuality Test

In [19]:
# If the p-value is less than 0.05 (or your chosen significance level), you reject H_0.
# This means that series X causes series Y, i.e., past values of X help predict future values of Y.
# If the p-value is greater than 0.05, you fail to reject H_0.
# This means that series X is not useful for predicting Y.

# Granger Test: Does stock cause sales?
# H0: Stock does NOT cause sales.
gc_test_stock_to_sales = grangercausalitytests(df_target[['Sales_Volume', 'Stock_Quantity']], maxlag=7)
# The maxlag=7 argument tests causality for up to 7 days of lag,
# which is useful for identifying potential weekly relationships.

# Granger Test: Do sales cause stock?
# H0: Sales do NOT cause stock.
gc_test_sales_to_stock = grangercausalitytests(df_target[['Stock_Quantity', 'Sales_Volume']], maxlag=7)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.2872  , p=0.1313  , df_denom=362, df_num=1
ssr based chi2 test:   chi2=2.3061  , p=0.1289  , df=1
likelihood ratio test: chi2=2.2989  , p=0.1295  , df=1
parameter F test:         F=2.2872  , p=0.1313  , df_denom=362, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.4279  , p=0.2412  , df_denom=359, df_num=2
ssr based chi2 test:   chi2=2.8955  , p=0.2351  , df=2
likelihood ratio test: chi2=2.8841  , p=0.2364  , df=2
parameter F test:         F=1.4279  , p=0.2412  , df_denom=359, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.0560  , p=0.3678  , df_denom=356, df_num=3
ssr based chi2 test:   chi2=3.2304  , p=0.3574  , df=3
likelihood ratio test: chi2=3.2161  , p=0.3595  , df=3
parameter F test:         F=1.0560  , p=0.3678  , df_denom=356, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.9131  , p=0.4563  

In [20]:
# Analyze the results of the "stock to sales" test:
print("Granger Test: Does stock cause sales?")
for lag in range(1, 8):
    p_value = gc_test_stock_to_sales[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

# Analyze the results of the "sales to stock" test:
print("\nGranger Test: Do sales cause stock?")
for lag in range(1, 8):
    p_value = gc_test_sales_to_stock[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

Granger Test: Does stock cause sales?
Lag 1: p-value = 0.1313
Lag 2: p-value = 0.2412
Lag 3: p-value = 0.3678
Lag 4: p-value = 0.4563
Lag 5: p-value = 0.5900
Lag 6: p-value = 0.7233
Lag 7: p-value = 0.8066

Granger Test: Do sales cause stock?
Lag 1: p-value = 0.2485
Lag 2: p-value = 0.5019
Lag 3: p-value = 0.5561
Lag 4: p-value = 0.4602
Lag 5: p-value = 0.4873
Lag 6: p-value = 0.6594
Lag 7: p-value = 0.7467


#### Summary:  

Since all p-values are **greater than 0.05**, the test result is not significant.  

This means that, based on the data and the test performed, there is no statistical evidence to conclude that past values of one time series help predict future values of the other.  
In other words, **no Granger causality relationship** was detected between the two variables at any of the tested lags.

---

# **Analytical Value**
This prepared dataset enables:
- Trend analysis and seasonal pattern identification
- Correlation analysis between inventory levels and sales performance
- Forecasting model development for both inventory and demand planning
- Anomaly detection in stock movements and sales patterns

In [21]:
# Select numeric columns (int, float)
numeric_cols = df_ts.select_dtypes(include=[np.number]).columns.tolist()

# Select non-numeric columns (exclude numbers and datetime objects)
non_numeric_cols = df_ts.select_dtypes(exclude=[np.number, 'datetime']).columns.tolist()

In [22]:
# Calculate the correlation matrix for all numeric columns
correlation_matrix = df_ts[numeric_cols].corr()

In [23]:
# Create a heatmap visualization of the correlation matrix
fig_corr = px.imshow(correlation_matrix,
                    title='Correlation Matrix - Numeric Variables',
                    color_continuous_scale='RdBu_r',  # Red-Blue reversed color scale
                    aspect="auto",                   # Automatic aspect ratio
                    text_auto=True,                  # Display correlation values on cells
                    zmin=-1, zmax=1)                 # Fix color scale from -1 to +1

# Adjust the figure dimensions
fig_corr.update_layout(width=800, height=800)

# Display the interactive heatmap
fig_corr.show()

In [24]:
# Extract specific correlations with the target variables
correlation_with_targets = correlation_matrix[target_columns].drop(target_columns)

In [25]:
# Create a bar chart for correlations with Sales_Volume
fig_sales = px.bar(correlation_with_targets['Sales_Volume'].sort_values(),
                  title='Correlation with Sales_Volume',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['Sales_Volume'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_sales.update_layout(showlegend=False)

# Display the interactive bar chart
fig_sales.show()

In [26]:
# Create a bar chart for correlations with Stock_Quantity
fig_stock = px.bar(correlation_with_targets['Stock_Quantity'].sort_values(),
                  title='Correlation with Stock_Quantity',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['Stock_Quantity'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_stock.update_layout(showlegend=False)

# Display the interactive bar chart
fig_stock.show()

**Summary:**

An analysis of the main correlation pairs reveals that most variable correlations are very weak, with two notable exceptions:

**Moderate Negative Correlations:**
*   **Days_For_Expiration & Delivery_Lag (-0.48):** A moderate negative correlation suggests products with a shorter shelf life (lower Days_For_Expiration) are shipped faster (higher Delivery_Lag).
*   **Inventory_Turnover_Rate & Stock_Coverage_Days (-0.48):** A logical negative correlation where a higher turnover rate (faster selling inventory) results in fewer days of stock coverage.

**Key Weak/Non-Existent Correlations:**
*   **Reorder_Level & Reorder_Quantity (~0.03):** The near-zero correlation is surprising, indicating the reorder trigger point has no linear relationship with the order quantity.
*   **Sales_Volume & Other Variables (~0.03):** Sales volume shows almost no linear correlation with any other supply chain metric (e.g., Stock_Quantity, Reorder_Level), confirming its unpredictability.

**In short:** The data shows mostly weak linear relationships. The most significant findings are the two moderate negative correlations and the notable lack of expected correlations, especially between reorder level/quantity, stock/other and sales volume/other metrics.

---

In [27]:
# Create a list of predictor variables by removing target columns from numeric columns
predictors_numeric = list(set(numeric_cols) - set(target_columns))

# Create a multi-panel histogram plot for numeric variable distributions
fig_dist = make_subplots(rows=len(predictors_numeric), cols=1,
                        subplot_titles=predictors_numeric,
                        vertical_spacing=0.05)  # Reduced spacing between subplots

# Add a histogram for each numeric predictor variable
for i, col in enumerate(predictors_numeric, 1):
    fig_dist.add_trace(go.Histogram(x=df_ts[col], name=col, nbinsx=50),  # 50 bins for detail
                      row=i, col=1)

# Adjust layout dimensions and title
fig_dist.update_layout(height=300*len(predictors_numeric),  # Dynamic height based on number of variables
                      title_text="Distribution of Numeric Predictor Variables",
                      showlegend=False)  # Legend not needed for individual histograms

# Display the interactive histogram matrix
fig_dist.show()

**Summary:**

**Distribution Analysis**

*   **Unit_Price:** Strongly right-skewed. Most products have a low price, with frequency dropping sharply as price increases. This is common, with most sales coming from low-cost items.
*   **Delivery_Lag:** Also strongly right-skewed. Most deliveries have very short or negligible delays, with a long tail of infrequent, larger delays.
*   **Inventory_Turnover_Rate:** Relatively uniform distribution. The frequency of values is consistent across the range, suggesting no single predominant turnover rate.
*   **Reorder_Level:** Multimodal distribution with several peaks. This indicates there isn't one dominant reorder level, but several common ones, likely reflecting different product categories or policies.
*   **Stock_Coverage_Days:** Right-skewed. Most items have short stock coverage, while only a few have very long coverage, which is expected to avoid overstocking.
*   **Days_For_Expiration:** The distribution is strongly right-skewed (positive skew). Most products have a short remaining shelf life, while a minority have a very long remaining shelf life.
*   **Reorder_Quantity:** Multimodal, with several peaks and valleys. Similar to Reorder_Level, this suggests the reorder quantity is not uniform but clusters around different common values.

---

In [28]:
# Scatter plots of the predictors most correlated with the targets
# Find the 4 variables most correlated with each target
top_sales_corr = correlation_with_targets['Sales_Volume'].abs().nlargest(4).index.tolist()
top_stock_corr = correlation_with_targets['Stock_Quantity'].abs().nlargest(4).index.tolist()

# Scatter plots for Sales_Volume
if top_sales_corr:
    fig_scatter_sales = make_subplots(rows=2, cols=2,
                                    subplot_titles=[f'{col} vs Sales_Volume' for col in top_sales_corr])
    
    for i, col in enumerate(top_sales_corr, 1):
        fig_scatter_sales.add_trace(go.Scatter(x=df_ts[col], y=df_ts['Sales_Volume'],
                                              mode='markers', name=col),
                                  row=(i+1)//2, col=(i % 2)+1)
    
    fig_scatter_sales.update_layout(height=600, title_text="Relationship with Sales_Volume")
    fig_scatter_sales.show()

**Summary:**

**Plot Analysis**

The visual analysis of all four plots leads to the same conclusion. In each one, the points are scattered randomly, forming a cloud without any clear trend or pattern.

*   **Inventory_Turnover_Rate vs. Sales_Volume (top left):** The points are horizontally and widely dispersed, indicating no linear relationship.
*   **Days_For_Expiration vs. Sales_Volume (top right):** The points are completely scattered, showing product shelf life is unrelated to sales volume.
*   **Reorder_Quantity vs. Sales_Volume (bottom left):** The stock reorder quantity does not appear to be related to sales volume.
*   **Delivery_Lag vs. Sales_Volume (bottom right):** The delivery delay shows no linear correlation with sales volume.

**Conclusion**
These scatter plots confirm previous analyses. The lack of a visual pattern demonstrates that **Sales Volume** has no significant linear correlation with turnover rate, product expiration, reorder quantity, or delivery lag.

This reinforces the idea that sales volume in this time series is unpredictable and appears to be influenced by random factors or variables not included in this analysis.

---

In [29]:
# Scatter plots for Stock_Quantity
if top_stock_corr:
    fig_scatter_stock = make_subplots(rows=2, cols=2,
                                    subplot_titles=[f'{col} vs Stock_Quantity' for col in top_stock_corr])
    
    for i, col in enumerate(top_stock_corr, 1):
        fig_scatter_stock.add_trace(go.Scatter(x=df_ts[col], y=df_ts['Stock_Quantity'],
                                              mode='markers', name=col),
                                  row=(i+1)//2, col=(i % 2)+1)
    
    fig_scatter_stock.update_layout(height=600, title_text="Relationship with Stock_Quantity")
    fig_scatter_stock.show()

**Summary:**

**Plot Analysis**

*   **Unit_Price vs. Stock_Quantity (top left):** The points are horizontally and widely dispersed, showing no clear trend. This indicates no linear relationship between a product's unit price and the stock quantity maintained.
*   **Inventory_Turnover_Rate vs. Stock_Quantity (top right):** This is the most interesting plot. Most points are concentrated in the upper-left corner, where Stock_Quantity is low and Inventory_Turnover_Rate is high. This suggests that for most items, a small stock is associated with a high turnover rate. The absence of a uniform cloud shows that, while not a perfect linear correlation, stock quantity does have a relationship with turnover.
*   **Reorder_Quantity vs. Stock_Quantity (bottom left):** The points are scattered randomly without any visible pattern. This shows no linear relationship between the reorder quantity and the current stock quantity.
*   **Days_For_Expiration vs. Stock_Quantity (bottom right):** The distribution of points is completely random, indicating no linear relationship between a product's shelf life and the stock quantity.

**Conclusion**
This analysis makes it clear that **Stock Quantity** does not have a simple linear relationship with most other variables, such as Unit_Price, Reorder_Quantity, and Days_For_Expiration. The notable exception is its relationship with **Inventory_Turnover_Rate**, where the data suggests that products with lower stock levels tend to have higher turnover rates—a logical finding for supply chain management.

---

In [30]:
# 8. Analysis of categorical variables
print("\n=== CATEGORICAL VARIABLE ANALYSIS ===")
for col in non_numeric_cols:
    print(f"\n{col}:")
    print(f"Number of categories: {df_ts[col].nunique()}")
    print(f"Unique values: {df_ts[col].unique()}")
    print(f"Count by category:")
    print(df_ts[col].value_counts())


=== CATEGORICAL VARIABLE ANALYSIS ===

Product_Name:
Number of categories: 121
Unique values: ['Arabica Coffee' 'Spinach' 'Peach' 'Corn Oil' 'Cheddar Cheese'
 'Feta Cheese' 'Plum' 'Egg (Turkey)' 'Swiss Cheese' 'Heavy Cream'
 'Blueberries' 'Rice Flour' 'Robusta Coffee' 'Eggplant' 'Bell Pepper'
 'Kale' 'Mango' 'Jasmine Rice' 'Olive Oil' 'Green Coffee' 'Whipped Cream'
 'Chocolate Biscuit' 'Cucumber' 'Cod' 'Powdered Sugar' 'Oatmeal Biscuit'
 'Herbal Tea' 'Peas' 'Apple' 'Digestive Biscuit' 'Egg (Goose)'
 'Green Beans' 'White Sugar' 'Strawberries' 'Cauliflower' 'Garlic'
 'Tomato' 'Halibut' 'Tuna' 'Arborio Rice' 'Watermelon' 'Canola Oil'
 'Pineapple' 'Vanilla Biscuit' 'Black Coffee' 'Papaya' 'Coconut Sugar'
 'Orange' 'Broccoli' 'White Bread' 'Almond Flour' 'Carrot'
 'Whole Wheat Flour' 'Salmon' 'Black Rice' 'Rye Bread' 'Palm Oil'
 'Short Grain Rice' 'Tilapia' 'Green Tea' 'Apricot' 'Mackerel' 'Grapes'
 'Banana' 'Egg (Chicken)' 'Vegetable Oil' 'Parmesan Cheese'
 'Sourdough Bread' 'Egg (Duck)' 

**Summary:**

**Stock Status Analysis**

Based on data, the total number of products analyzed is **990** (499 Expired + 424 Safe + 67 Nearing).  
Converting these numbers to percentages provides a clearer picture:

- **Expired:** 499 products, representing **50.4%** of the stock.
- **Nearing Expiry:** 67 products, corresponding to **6.8%** of the total.
- **Safe:** 424 products, or **42.8%** of the stock.

**Conclusion:**  
The results are **extremely concerning**. The proportion of expired products exceeds that of products in safe selling condition. This suggests **critical issues in inventory management**, such as overstocking, poor demand forecasting, or ineffective stock rotation practices.

---

In [31]:
# Box plots for categorical variables vs targets
for cat_col in non_numeric_cols:
    if df_ts[cat_col].nunique() <= 10:  # Only plot if there are up to 10 categories
        fig_box = px.box(df_ts, x=cat_col, y='Sales_Volume',
                        title=f'Sales_Volume by {cat_col}')
        fig_box.update_layout(height=400, width=900)
        fig_box.show()
        
        fig_box2 = px.box(df_ts, x=cat_col, y='Stock_Quantity',
                         title=f'Stock_Quantity by {cat_col}')
        fig_box2.update_layout(height=400, width=900)
        fig_box2.show()

**Summary:**

**1. Sales Volume by Category and Status**

- **Sales_Volume by Category:**  
  The boxplots and medians for all product categories (Beverages, Fruits & Vegetables, Oils & Fats, etc.) are very similar.  
  **Conclusion:** Sales volume distribution is consistent across categories. No category sells significantly more or less than others.

- **Sales_Volume by Expiry Status:**  
  The boxplots for **Safe**, **Nearing Expiry**, and **Expired** products are nearly identical, with matching medians and quartile ranges.  
  **Conclusion:** A critical finding. Sales volume does not decrease as products approach or pass their expiration date. This reinforces the earlier conclusion that sales are largely random and uninfluenced by product status.

- **Sales_Volume by Product Status:**  
  Sales distributions for **Discontinued**, **Backordered**, and **Active** products are very similar.  
  **Conclusion:** Surprisingly, discontinued or backordered products show sales volumes distributed similarly to active ones. This may indicate issues with status categorization, high sales unpredictability, or a discontinuation policy unrelated to sales performance.

**2. Stock Quantity by Category and Expiry Status**

- **Stock_Quantity by Category:**  
  Stock quantity varies more across categories than sales. For example, **Seafood** has a significantly higher median and interquartile range than **Beverages** or **Grains & Pulses**.  
  **Conclusion:** Inventory policies or realities differ by product category, which makes sense (e.g., seafood may require different stock levels than beverages).

- **Stock_Quantity by Expiry Status:**  
  Medians for **Safe** and **Expired** products are very close, but the median for **Nearing Expiry** products is considerably lower.  
  **Conclusion:** Lower stock for nearing-expiry items may indicate active management and sales. However, the high median stock for **Expired** products (equal to Safe) is a major red flag: large quantities are expiring in stock instead of being sold in time.

**Overall Summary:**  
The boxplot analysis deepens previous findings. Sales unpredictability is a recurring theme, confirmed by similar distributions across categories, expiry status, and product status. Inventory behavior is more logical and varied across categories but reveals a critical management issue: significant volumes of high-quantity stock are expiring rather than being sold.

---

In [32]:
# Outlier analysis using box plots

cols = 4  # Number of columns in the grid
rows = (len(predictors_numeric) + cols - 1) // cols  # Calculate rows needed (rounded up)

# Create subplot grid
fig_outliers = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=predictors_numeric,  # Title for each subplot
    vertical_spacing=0.1,               # Space between rows
    horizontal_spacing=0.2,             # Space between columns
)

# Add boxplots for each numeric variable
for i, var in enumerate(predictors_numeric):
    row = i // cols + 1  # Calculate row position (integer division)
    col = i % cols + 1   # Calculate column position (modulo operation)
    
    fig_outliers.add_trace(
        go.Box(y=df_ts[var], name=var, boxpoints='outliers'),  # Show only outliers
        row=row, col=col
    )

# Adjust layout
fig_outliers.update_layout(
    height=600,
    width=900,
    title_text="Outlier Analysis - Numeric Variables",
    showlegend=False  # Hide legend as each box is labeled
)

# Remove x-axis labels for cleaner look
fig_outliers.update_xaxes(showticklabels=False)

fig_outliers.show()

**Summary:**

**Variable Analysis**

*   **Reorder_Level and Reorder_Quantity:** Both variables show a relatively uniform distribution, with most data concentrated within the boxes and whiskers, indicating few outliers. Any outliers are close to the upper limit and not extreme.
*   **Delivery_Lag:** The boxplot for delivery time is slightly more spread out, but most data is still well concentrated. However, some isolated points (outliers) above the upper limit suggest a few deliveries took significantly longer than usual.
*   **Stock_Coverage_Days:** This variable has considerable dispersion. The box is relatively small, but the whiskers are long, with many isolated data points far above the main distribution. This suggests many outliers, indicating exceptionally high stock coverage for some items.
*   **Days_For_Expiration:** The distribution for days until expiration shows significant spread, similar to Stock_Coverage_Days, with multiple upper outliers. This suggests some products have a much longer shelf life than the rest of the stock.
*   **Inventory_Turnover_Rate:** The inventory turnover rate has a well-centered distribution with few outliers, indicating a consistent turnover rate for most products without many extreme values.
*   **Unit_Price:** Unit price shows the highest number of outliers. The box is very small and located at the bottom, indicating the vast majority of items have a low price. However, a large number of isolated points above it point to the existence of some products with prices much higher than average, which is common and may represent high-value items.

---

# Summary of Exploratory Data Analysis (EDA)

This report summarizes the Exploratory Data Analysis (EDA) conducted for the **"Intelligent System for Supply Chain Management"** project, which aims to **reduce overstock by 20%** within 6 months.  
The analysis focused on the inventory optimization variable (`Stock_Quantity`) and the demand forecasting variable (`Sales_Volume`).

---

#### Key Findings from Time Series Analysis

The analysis of the `Sales_Volume` and `Stock_Quantity` time series revealed the following points:

* **Weekly Seasonality:**  
Both series, especially sales volume, show a consistent weekly seasonal pattern with peaks and troughs corresponding to specific days of the week.

* **Volatility:**  
The series are highly volatile, and a significant outlier was identified in November 2024, possibly related to a promotional event.

* **Stationarity:**  
The series do not show a long-term trend and appear to be stationary.

---

#### Correlation Analysis

The correlation analysis between the variables shows predominantly weak linear relationships:

* **Moderate Negative Correlations:**  
A moderate correlation was observed between `Days_For_Expiration` and `Delivery_Lag` (-0.48), indicating that products with shorter shelf lives tend to be delivered more quickly.  
Another similar correlation (-0.48) exists between `Inventory_Turnover_Rate` and `Stock_Coverage_Days`, showing that products with a higher turnover rate have fewer days of stock coverage.

* **Lack of Sales Relationship:**  
Sales volume (`Sales_Volume`) shows no significant linear correlation with any other supply chain metric, such as `Stock_Quantity`, `Reorder_Level`, or `Reorder_Quantity`.  
This reinforces the unpredictability of sales behavior.

---

#### Critical Inventory Management Problem

The stock analysis by expiration status revealed a critical flaw:

* The median stock quantity for **Expired** products is equal to that of products considered **Safe**.  
This is a red flag that large volumes of products are expiring in stock instead of being sold in time.

---

#### Overall Summary

The exploratory analysis suggests that sales volume in the current time series is unpredictable and does not have a clear linear relationship with other system metrics.  
On the other hand, stock behavior presents a critical management issue, with significant volumes of products expiring instead of being sold, which represents an opportunity for system improvement.